In [7]:
# === 競賽資料時間截斷處理 ===
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path

def load_condition_mapping():
    """載入加工條件對照表"""
    try:
        # 載入條件對照表
        condition_file = "/Users/benjamin/1132/11325/AI_Race/檔案環境設定總表.csv"
        if os.path.exists(condition_file):
            condition_df = pd.read_csv(condition_file, encoding='utf-8')
            print(f"✅ 載入條件對照表: {len(condition_df)} 筆記錄")
            
            # 轉換為字典格式
            condition_map = {}
            for _, row in condition_df.iterrows():
                # 取得日期欄位 (第一欄)
                date_key = str(row['日期']).strip()
                
                # 計算總加工時間
                total_time = 0
                stage_info = []
                
                # 第一段時間
                time1 = row.iloc[3] if pd.notna(row.iloc[3]) else 0  # 時間 (Hr) 欄位
                if time1 > 0:
                    total_time += time1
                    stage_info.append(f"第一段: {time1}H")
                
                # 第二段時間  
                time2 = row.iloc[6] if pd.notna(row.iloc[6]) else 0
                if time2 > 0:
                    total_time += time2
                    stage_info.append(f"第二段: {time2}H")
                
                # 第三段時間
                time3 = row.iloc[9] if pd.notna(row.iloc[9]) else 0
                if time3 > 0:
                    total_time += time3
                    stage_info.append(f"第三段: {time3}H")
                
                condition_map[date_key] = {
                    'total_hours': total_time,
                    'time1': time1,
                    'time2': time2, 
                    'time3': time3,
                    'stage_info': stage_info,
                    'temperature': row.iloc[11] if pd.notna(row.iloc[11]) else 'Unknown',  # 溫度欄位
                    'temp_control': row.iloc[10] if pd.notna(row.iloc[10]) else 'Unknown'  # 控溫欄位
                }
                
                print(f"📅 {date_key}: 總時間 {total_time}H - {', '.join(stage_info)}")
            
            return condition_map
        else:
            print(f"⚠️ 找不到條件對照表: {condition_file}")
            return {}
    except Exception as e:
        print(f"❌ 載入條件對照表失敗: {e}")
        return {}

def truncate_overtime_data(csv_file_path, condition_map):
    """
    根據加工條件截斷超時資料
    
    Parameters:
    - csv_file_path: CSV 檔案路徑
    - condition_map: 加工條件對照表
    
    Returns:
    - truncated_df: 截斷後的 DataFrame
    - truncation_info: 截斷資訊
    """
    
    # 載入原始資料
    df = pd.read_csv(csv_file_path, low_memory=False)
    filename = os.path.basename(csv_file_path)
    
    try:
        # 從檔名提取日期 (改進版本)
        date_str = filename.split('_')[1]  # 例如: 20200615
        condition = condition_map.get(date_str, {})
        
        if not condition:
            print(f"⚠️ 找不到 {filename} ({date_str}) 的加工條件，使用預設時間")
            total_hours = estimate_total_time_from_filename(filename)
        else:
            # 從條件對照表取得總加工時間
            total_hours = condition.get('total_hours', 0)
            stage_info = ', '.join(condition.get('stage_info', []))
            print(f"✅ {filename} ({date_str}): 找到條件 - {stage_info}")
            
            # 如果條件對照表沒有時間資訊，使用預設值
            if total_hours == 0:
                total_hours = estimate_total_time_from_filename(filename)
                print(f"⚠️ 條件對照表無時間資訊，使用預設: {total_hours}H")
        
        print(f"📋 {filename}: 計劃加工時間 = {total_hours} 小時")
        
        # 假設取樣頻率 (需要根據實際資料調整)
        if 'Time' in df.columns:
            # 使用 Time 欄位計算
            time_values = pd.to_numeric(df['Time'], errors='coerce')
            max_time_seconds = total_hours * 3600  # 轉換為秒
            
            # 找到截斷點
            valid_mask = time_values <= max_time_seconds
            truncate_index = valid_mask.sum()
            
        else:
            # 使用行數估算 (假設固定取樣頻率)
            sampling_rate_per_hour = estimate_sampling_rate(df, total_hours)
            expected_rows = int(total_hours * sampling_rate_per_hour)
            truncate_index = min(expected_rows, len(df))
        
        # 執行截斷
        if truncate_index < len(df):
            truncated_df = df.iloc[:truncate_index].copy()
            truncation_info = {
                'truncated': True,
                'original_rows': len(df),
                'truncated_rows': truncate_index,
                'removed_rows': len(df) - truncate_index,
                'planned_hours': total_hours,
                'truncate_reason': 'overtime_data_removal'
            }
            print(f"✂️  截斷 {filename}: {len(df)} → {truncate_index} 行 (移除 {len(df) - truncate_index} 行)")
        else:
            truncated_df = df.copy()
            truncation_info = {
                'truncated': False,
                'original_rows': len(df),
                'planned_hours': total_hours,
                'truncate_reason': 'no_overtime_data'
            }
            print(f"✅ {filename}: 無需截斷 ({len(df)} 行)")
        
        return truncated_df, truncation_info
        
    except Exception as e:
        print(f"❌ 處理 {filename} 時發生錯誤: {e}")
        return df, {'truncated': False, 'reason': f'error: {e}'}

def estimate_total_time_from_filename(filename):
    """從檔名推估總加工時間"""
    
    # 手動對照表 (根據檔案環境設定總表)
    time_mapping = {
        '20200615': 5.0,   # 2000rpm, 5H
        '20200616': 5.0,   # 1000rpm, 5H  
        '20200617': 5.0,   # 1000rpm 2.5H + 2000rpm 2.5H
        '20200618': 5.0,   # 2000rpm 2.5H + 1000rpm 2.5H
        '20200701': 5.0,   # 2000rpm, 5H
        '20200702': 5.0,   # 1000rpm, 5H
        '20200703': 5.0,   # 1000rpm 2.5H + 2000rpm 2.5H
        '20200706': 5.0,   # 2000rpm 2.5H + 1000rpm 2.5H
        '20200708': 6.0,   # 2000rpm, 6H (變溫)
        '20200709': 6.0,   # 1000rpm, 6H (變溫)
        '20200710': 6.0,   # 1000rpm 3H + 2000rpm 3H (變溫)
        '20200713': 6.0,   # 2000rpm 3H + 1000rpm 3H (變溫)
        '20200715': 6.0,   # 1800rpm 2.5H + 停機 1H + 1200rpm 2.5H
        # ... 其他日期
    }
    
    date_part = filename.split('_')[1] if '_' in filename else ''
    return time_mapping.get(date_part, 5.0)  # 預設 5 小時

def estimate_sampling_rate(df, total_hours):
    """估算取樣頻率"""
    
    # 方法1: 使用 Time 欄位
    if 'Time' in df.columns:
        time_values = pd.to_numeric(df['Time'], errors='coerce').dropna()
        if len(time_values) > 1:
            max_time_seconds = time_values.max()
            sampling_rate = len(time_values) / (max_time_seconds / 3600)  # 每小時樣本數
            return sampling_rate
    
    # 方法2: 假設固定取樣頻率
    # 根據現有資料推估 (例如: 每分鐘1筆)
    estimated_rate_per_hour = 60  # 每小時60筆
    return estimated_rate_per_hour

def batch_truncate_files(train_dir, condition_map, output_dir=None):
    """批次處理所有檔案的時間截斷"""
    
    if output_dir:
        Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    train_files = sorted(glob.glob(os.path.join(train_dir, "*.csv")))
    truncation_summary = []
    
    print(f"=== 📂 批次截斷處理: {len(train_files)} 個檔案 ===")
    
    for file_path in train_files:
        filename = os.path.basename(file_path)
        
        # 執行截斷
        truncated_df, info = truncate_overtime_data(file_path, condition_map)
        
        # 儲存處理後的檔案
        if output_dir:
            output_path = os.path.join(output_dir, filename)
            truncated_df.to_csv(output_path, index=False)
        
        # 記錄處理結果
        info['filename'] = filename
        truncation_summary.append(info)
    
    # 顯示處理摘要
    summary_df = pd.DataFrame(truncation_summary)
    
    print(f"\n=== 📊 截斷處理摘要 ===")
    truncated_count = summary_df['truncated'].sum()
    total_removed = summary_df['removed_rows'].sum() if 'removed_rows' in summary_df.columns else 0
    
    print(f"📋 處理檔案總數: {len(summary_df)}")
    print(f"✂️  截斷檔案數: {truncated_count}")
    print(f"📉 總移除行數: {total_removed:,}")
    print(f"💾 平均移除比例: {(total_removed / summary_df['original_rows'].sum() * 100):.1f}%")
    
    # 顯示需要截斷的檔案
    if truncated_count > 0:
        truncated_files = summary_df[summary_df['truncated'] == True]
        print(f"\n⚠️  需要截斷的檔案:")
        for _, row in truncated_files.iterrows():
            removal_pct = (row['removed_rows'] / row['original_rows'] * 100)
            print(f"   {row['filename']}: 移除 {row['removed_rows']} 行 ({removal_pct:.1f}%)")
    
    return summary_df

# === 使用範例 ===
if __name__ == "__main__":
    
    # 載入條件對照表
    condition_map = load_condition_mapping()
    
    # 設定路徑
    TRAIN_DIR = "/Users/benjamin/1132/11325/AI_Race/2025_dataset_0806 3/train"
    OUTPUT_DIR = "/Users/benjamin/1132/11325/AI_Race/truncated_data"
    
    # 執行批次截斷
    summary = batch_truncate_files(TRAIN_DIR, condition_map, OUTPUT_DIR)
    
    # 儲存處理報告
    report_file = f"{OUTPUT_DIR}/truncation_report.csv"
    summary.to_csv(report_file, index=False)
    print(f"\n📄 處理報告已儲存: {report_file}")

✅ 載入條件對照表: 57 筆記錄
❌ 載入條件對照表失敗: '>' not supported between instances of 'str' and 'int'
=== 📂 批次截斷處理: 43 個檔案 ===
⚠️ 找不到 _20200615_GV1-1203_2000rpm_XZ-5m-min_5H(wAC-from0-25C).csv (20200615) 的加工條件，使用預設時間
📋 _20200615_GV1-1203_2000rpm_XZ-5m-min_5H(wAC-from0-25C).csv: 計劃加工時間 = 5.0 小時
✅ _20200615_GV1-1203_2000rpm_XZ-5m-min_5H(wAC-from0-25C).csv: 無需截斷 (515 行)
⚠️ 找不到 _20200616_GV1-1203_1000rpm_XZ-5m-min_5H(wAC-from0-25C).csv (20200616) 的加工條件，使用預設時間
📋 _20200616_GV1-1203_1000rpm_XZ-5m-min_5H(wAC-from0-25C).csv: 計劃加工時間 = 5.0 小時
✅ _20200616_GV1-1203_1000rpm_XZ-5m-min_5H(wAC-from0-25C).csv: 無需截斷 (605 行)
⚠️ 找不到 _20200617_GV1-1203_1k+2krpm_XZ-5m-min_5H(wAC-from0-25C).csv (20200617) 的加工條件，使用預設時間
📋 _20200617_GV1-1203_1k+2krpm_XZ-5m-min_5H(wAC-from0-25C).csv: 計劃加工時間 = 5.0 小時
✅ _20200617_GV1-1203_1k+2krpm_XZ-5m-min_5H(wAC-from0-25C).csv: 無需截斷 (607 行)
⚠️ 找不到 _20200618_GV1-1203_2k+1krpm_XZ-5m-min_5H(wAC-from0-25C).csv (20200618) 的加工條件，使用預設時間
📋 _20200618_GV1-1203_2k+1krpm_XZ-5m-min_5H(wAC-from0-25C).csv: 計劃加

In [8]:
# === 📁 資料路徑導覽總結 ===

import os
import pandas as pd

def show_data_structure():
    """顯示整理後的資料結構"""
    
    print("🗂️  AI競賽資料結構導覽")
    print("=" * 50)
    
    # 原始訓練資料
    train_dir = "/Users/benjamin/1132/11325/AI_Race/2025_dataset_0806 3/train"
    train_files = len([f for f in os.listdir(train_dir) if f.endswith('.csv')])
    print(f"📂 原始訓練資料: {train_dir}")
    print(f"   └── 檔案數量: {train_files} 個 CSV 檔案")
    
    # 截斷處理後資料
    truncated_dir = "/Users/benjamin/1132/11325/AI_Race/truncated_data"
    if os.path.exists(truncated_dir):
        truncated_files = len([f for f in os.listdir(truncated_dir) if f.endswith('.csv') and f != 'truncation_report.csv'])
        print(f"📂 時間截斷資料: {truncated_dir}")
        print(f"   ├── 處理檔案: {truncated_files} 個")
        print(f"   └── 處理報告: truncation_report.csv")
        
        # 讀取處理報告
        if os.path.exists(f"{truncated_dir}/truncation_report.csv"):
            report = pd.read_csv(f"{truncated_dir}/truncation_report.csv")
            truncated_count = report['truncated'].sum()
            total_files = len(report)
            print(f"   └── 截斷統計: {truncated_count}/{total_files} 檔案需要截斷")
    
    # 預測結果資料
    pred_dir = "/Users/benjamin/1132/11325/AI_Race/preds_out"
    if os.path.exists(pred_dir):
        pred_files = len([f for f in os.listdir(pred_dir) if f.endswith('.csv')])
        print(f"📂 預測結果資料: {pred_dir}")
        print(f"   └── 預測檔案: {pred_files} 個")
    
    # 條件對照表
    condition_file = "/Users/benjamin/1132/11325/AI_Race/檔案環境設定總表.csv"
    if os.path.exists(condition_file):
        condition_df = pd.read_csv(condition_file)
        print(f"📋 加工條件對照表: 檔案環境設定總表.csv")
        print(f"   └── 條件記錄: {len(condition_df)} 筆")
    
    print("\n🎯 建議使用路徑:")
    print(f"   • 訓練資料: {truncated_dir} (已處理時間截斷)")
    print(f"   • 原始資料: {train_dir} (未處理)")
    print(f"   • 預測輸出: {pred_dir}")
    print(f"   • 條件對照: {condition_file}")

# 執行路徑導覽
show_data_structure()

🗂️  AI競賽資料結構導覽
📂 原始訓練資料: /Users/benjamin/1132/11325/AI_Race/2025_dataset_0806 3/train
   └── 檔案數量: 43 個 CSV 檔案
📂 時間截斷資料: /Users/benjamin/1132/11325/AI_Race/truncated_data
   ├── 處理檔案: 43 個
   └── 處理報告: truncation_report.csv
   └── 截斷統計: 0/43 檔案需要截斷
📋 加工條件對照表: 檔案環境設定總表.csv
   └── 條件記錄: 57 筆

🎯 建議使用路徑:
   • 訓練資料: /Users/benjamin/1132/11325/AI_Race/truncated_data (已處理時間截斷)
   • 原始資料: /Users/benjamin/1132/11325/AI_Race/2025_dataset_0806 3/train (未處理)
   • 預測輸出: /Users/benjamin/1132/11325/AI_Race/preds_out
   • 條件對照: /Users/benjamin/1132/11325/AI_Race/檔案環境設定總表.csv
